In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

In [ ]:
df_train = pd.read_csv("./kaggle/train.csv")
df_test = pd.read_csv("./kaggle/test.csv")

display(df_train.head())
display(df_test.head())

# Exploratory Analysis

Распределим предикторы по типу данных - числовые значения (num_var) и категории (cat_var)

In [ ]:
num_var = ["SibSp", "Parch", "Age", "Fare"]
cat_var = ["Sex", "Embarked", "Pclass"]
target = "Survived"

In [ ]:
# Функции визуализации распределения значений в датасете

# Функция распределения числовых значений
def num_dist(data, var):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))

    sns.histplot(data=data, x=var, kde=True, ax=ax[0])
    sns.boxplot(data=data, x=var, ax=ax[1])
    ax[0].set_title(f"Гистограмма распределения {var}")
    ax[1].set_title(f"График распределения {var} ")

    plt.show()
    
# Функция распределения по категориям
def cat_dist(data, var):
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    df_train[var].value_counts().plot(kind="pie", explode=[0.05 for x in data[var].dropna().unique()], autopct='%1.1f%%', ax=ax[0], shadow=True)
    ax[0].set_title(f"Круговая диаграмма '{var}'")
    ax[0].set_ylabel('')

    count = sns.countplot(x=var, data=df_train, ax=ax[1])
    for bar in count.patches:
        count.annotate(format(bar.get_height()),
            (bar.get_x() + bar.get_width() / 2,
            bar.get_height()), ha='center', va='center',
            size=11, xytext=(0, 8),
            textcoords='offset points')
    ax[1].set_title(f" Столбчатая Диаграмма '{var}'")
    plt.show()

## Выживаемость

Посмотрим, сколько человек выжило/не выжило

In [ ]:
cat_dist(df_train, "Survived")

61,6% пассажиров не выжили. Только 342 из 891 людей из тренировочных данных удалось спастись. Попытаемся выявить факторы, влияющие на выживаемость

## Распределение предикторов

### Численные значения

In [ ]:
df_train[num_var].describe()

In [ ]:
for var in num_var:
    num_dist(df_train, var)

Большинство путешествовало без семей (показатель SibSp). Возраст колеблется от 0.42 до 80 лет, в среднем 29.7 лет (показатель Age). Некоторые пассажиры не платили за поездку.

### Категории

In [ ]:
for var in cat_var:
    cat_dist(df_train, var)

64.8% пассажиров - мужчины. Более 70% пассажиров зашли на борт в Саутгемптоне (порт S), почти 19% в Квинстауне (Q), и только 8.7% в Шербуре (C). Большинство ехало 3-м классом.

## Сравнение показателей у выживших и невыживших

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(20, 8))
ax = ax.flatten()

for i, var in enumerate(num_var+cat_var):
    if i < 4:
        sns.histplot(data=df_train, x=var, hue=target, kde=True, ax=ax[i])
    else:
        sns.countplot(data=df_train, x=var, hue=target, ax=ax[i])
    
    ax[i].set_title(f"{var}")
    
plt.subplots_adjust(hspace=0.5)
plt.show()

Результаты:
* Путешествующие без семьи имели большие шансы выжить.
* Выжили в основном дети и молодые люди, с увеличением возраста выживаемость уменьшается. 
* Процент выживаемости у женщин намного больше, чем у мужчин. 
* Пассажиры, зашедшие в Саутгемптоне, имеют наименьшую выживаемость. У остальных процент выживаемости намного выше. 
* Подавляющее большинство пассажиров 3 класса не выжило. Большая часть пассажирова 1 класса выжила.


In [ ]:
sns.violinplot(data=df_train, x="Sex", y="Age", hue="Survived", split=True)
plt.show()

Видно, что у мальчиков и женщин пожилого возраста выживаемость больше.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

for i, var in enumerate(["SibSp", "Parch"]):
    surv = sns.barplot(data=df_train, x=var, y=target, ax=ax[i], ci=None)
    for bar in surv.patches:
        surv.annotate(format("{:.3f}".format(bar.get_height())),
            (bar.get_x() + bar.get_width() / 2,
            bar.get_height()), ha='center', va='center',
            size=11, xytext=(0, 8),
            textcoords='offset points')
        
    ax[i].set_title(f"{var} Survival Rate")

График демонстрирует тенденцию к выживаемости у людей с небольшим количеством родственников-спутников. При этом наибольший процент не у одиноких пассажиров, а именно у пар и троек родственных пассажиров.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 6))

for i, pc in enumerate(sorted(df_train["Pclass"].unique())):
    sns.histplot(data=df_train[df_train["Pclass"]==pc], x="Fare", hue=target, kde=True, ax=ax[i])
    ax[i].set_title(f"Fare in Pclass {pc} Survival Rate")

У пассажиров, заплативших больше, прослеживается тенденция к большей выживаемости. Хотя гарантий тоже не прослеживается: многие пассажиры 2го класса, заплатившие больше всего, не выжили.

# Подбор параметров

##  PassengerId

Удалим ненужный параметр ID

In [ ]:
df_train_prep = df_train.drop("PassengerId", axis=1)
df_test_prep = df_test.drop("PassengerId", axis=1)

## Name

In [ ]:
df_train["Name"]

 В имени можно выделить титулы (Mr, Mrs, Miss, и т.д.) по наличию точки.

In [ ]:
df_train_prep["Title"] = df_train_prep["Name"].str.extract('([A-Za-z]+)\.')
df_test_prep["Title"] = df_test_prep["Name"].str.extract('([A-Za-z]+)\.')

df_train_prep["Title"].value_counts()

Объединим некоторые титулы по смыслу и важности в условии задачи

In [ ]:
def convert_title(title):
    if title in ["Ms", "Mile", "Miss"]:
        return "Miss"
    elif title in ["Mme", "Mrs"]:
        return "Mrs"
    elif title == "Mr":
        return "Mr"
    elif title == "Master":
        return "Master"
    else:
        return "Other"
        
df_train_prep["Title"] = df_train_prep["Title"].map(convert_title)
df_test_prep["Title"] = df_test_prep["Title"].map(convert_title)

df_train_prep["Title"].value_counts()

Теперь переменная имени нам больше не нужна

In [ ]:
df_train_prep = df_train_prep.drop("Name", axis=1)
df_test_prep = df_test_prep.drop("Name", axis=1)

## Ticket

In [ ]:
df_train_prep["Ticket"]

Номер каждого билета уникален, и никакой полезной информации из него получить нельзя, поэтому удалим данный параметр.

In [ ]:
df_train_prep = df_train_prep.drop("Ticket", axis=1)
df_test_prep = df_test_prep.drop("Ticket", axis=1)

Перед дальнейшим анализом заметим, что в датасетах присутствуют параметры с множеством значений Null. Это такие параметры, как Age и Cabin

In [ ]:
pd.DataFrame(data = [df_train.isna().sum()/df_train.shape[0]*100, df_test.isna().sum()/df_test.shape[0]*100], index=["Train Null (%)", "Test Null (%)"]).T.style.background_gradient(cmap='summer_r')

## Cabin

Полезной информации из номеров каюты, кажется, не достать, поэтому этот параметр тоже уберем

In [ ]:
df_train_prep = df_train_prep.drop("Cabin", axis=1)
df_test_prep = df_test_prep.drop("Cabin", axis=1)

## Age

Параметр возраст у многих пассажиров пропущен, хотя на основе предыдущего анализа выглядит довольно важным. Необходимо провести подстановку значений этого параметра. Для этого попробуем проверить корелляцию с параметром титула, выявленном ранее.

In [ ]:
sns.violinplot(data=df_train_prep, x="Title", y="Age")
plt.show()

На основе этих данных можно выявить вероятный возраст пассажира, принадлежащего к определенной группе

In [ ]:
df_train_prep.groupby('Title')['Age'].median()

Заполним пустые поля медианными значениями в каждой группе

In [ ]:
df_train_prep.loc[(df_train_prep.Age.isnull()) & (df_train_prep["Title"]=='Master'), 'Age'] = 4
df_train_prep.loc[(df_train_prep.Age.isnull()) & (df_train_prep["Title"]=='Miss'), 'Age'] = 21
df_train_prep.loc[(df_train_prep.Age.isnull()) & (df_train_prep["Title"]=='Mr'), 'Age'] = 30
df_train_prep.loc[(df_train_prep.Age.isnull()) & (df_train_prep["Title"]=='Mrs'), 'Age'] = 35
df_train_prep.loc[(df_train_prep.Age.isnull()) & (df_train_prep["Title"]=='Other'), 'Age'] = 47

In [ ]:
df_test_prep.loc[(df_test_prep.Age.isnull())&(df_test_prep["Title"]=='Master'), 'Age'] = 4
df_test_prep.loc[(df_test_prep.Age.isnull())&(df_test_prep["Title"]=='Miss'), 'Age'] = 21
df_test_prep.loc[(df_test_prep.Age.isnull())&(df_test_prep["Title"]=='Mr'), 'Age'] = 30
df_test_prep.loc[(df_test_prep.Age.isnull())&(df_test_prep["Title"]=='Mrs'), 'Age'] = 35
df_test_prep.loc[(df_test_prep.Age.isnull())&(df_test_prep["Title"]=='Other'), 'Age'] = 47

## Fare

У этого параметра тоже ест пропущенные значения, так что попробуем подставить и их

In [ ]:
df_train_prep.corr()["Fare"].sort_values(ascending=False)

Параметр Pclass имеет неплохую отрецательную корелляцию с Fare. Используем этот факт

In [ ]:
df_test_prep[df_test_prep["Fare"].isna()]

Pclass = 3

In [ ]:
df_test_prep.Fare.fillna(df_train.groupby("Pclass").median()["Fare"][3], inplace=True)

## Категории

In [ ]:
df_train_prep = pd.get_dummies(df_train_prep, prefix=["Sex", "Embarked", "Title"])
df_test_prep = pd.get_dummies(df_test_prep, prefix=["Sex", "Embarked", "Title"])

# Построение модели

In [ ]:
X_train = df_train_prep.drop("Survived", axis=1)
y_train = df_train_prep.Survived

X_test = df_test_prep.copy()

## Скейлинг параметров

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
classifiers = {
    "KNN": KNeighborsClassifier(), 
    "LR": LogisticRegression(max_iter=1000), 
    "DT": DecisionTreeClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "MLP": MLPClassifier(max_iter=1000),
    "XGB": XGBClassifier(),
    "LGBM": LGBMClassifier()
}

results = dict()
for name, clf in classifiers.items():
    model = clf
    cv_results = cross_validate(
        model, X_train_scaled, y_train, cv=5,
        scoring=('accuracy')
    )

    results[name] = cv_results['test_score'].mean()
    
results

In [ ]:
import warnings
warnings.filterwarnings("ignore")

lgbm = LGBMClassifier(random_state=0)
params = {
    "boosting_type": ["gbdt", "dart", "goss"],
    "learning_rate": [0.1, 0.05, 0.01],
    "n_estimators": [10, 50, 100, 300]
}
clf = GridSearchCV(lgbm, params, cv=10)
clf.fit(X_train_scaled, y_train)
print("Best hyperparameter:", clf.best_params_)

In [ ]:
y_pred = clf.predict(X_train_scaled)
print(classification_report(y_train, y_pred))

In [ ]:
y_pred = clf.predict(X_test_scaled)

submission = pd.DataFrame({
         "PassengerId": df_test["PassengerId"],
         "Survived": y_pred
     })
submission.to_csv('prediction.csv', index=False)